# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# Helper functions

def get_simple_mask_vertices(image_shape):

    return np.array([[
        (400, 300), (50, image_shape[0]), (image_shape[1] - 50, image_shape[0]), (image_shape[1] - 400, 300)
    ]])

def get_image_contours(image):

    return cv2.adaptiveThreshold(image, maxValue=255, adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 thresholdType=cv2.THRESH_BINARY_INV, blockSize=7, C=5)

def draw_lane(image, left_line, right_line):

    vertices = np.array(
        [
            [
                (left_line[0], left_line[1]),
                (left_line[2], left_line[3]),
                (right_line[2], right_line[3]),
                (right_line[0], right_line[1])
            ]
        ])


    cv2.fillPoly(image, vertices, color=[0, 255, 0])


def get_xyz_grayscale(image):

    changed = cv2.cvtColor(image, cv2.COLOR_RGB2XYZ)
    return changed[:,:,2]


def get_xyz_space_contours(image):

    grayscale_image = get_xyz_grayscale(image)
    blurred_image = gaussian_blur(grayscale_image, 5)
    contours_image = get_image_contours(blurred_image)

    return contours_image


def is_line_left_lane_candidate(line, image_shape):

    x1, y1, x2, y2 = line

    slope_degrees = np.rad2deg(np.arctan2(y2 - y1, x2 - x1))
    is_left_lane_slope = -45 < slope_degrees < -15

    max_x = 0.5 * image_shape[1]
    is_line_left_of_vehicle = x1 < max_x and max_x

    return is_left_lane_slope and is_line_left_of_vehicle


def is_line_right_lane_candidate(line, image_shape):

    x1, y1, x2, y2 = line

    slope_degrees = np.rad2deg(np.arctan2(y2 - y1, x2 - x1))
    is_right_lane_slope = 15 < slope_degrees < 45

    min_x = 0.5 * image_shape[1]
    is_line_right_of_vehicle = x1 > min_x and x2 > min_x

    return is_right_lane_slope and is_line_right_of_vehicle


def get_left_road_lane_candidates(lines, image_shape):

    return [line for line in lines if is_line_left_lane_candidate(line, image_shape)]


def get_right_road_lane_candidates(lines, image_shape):

    return [line for line in lines if is_line_right_lane_candidate(line, image_shape)]


def get_filtered_lines(lines):
    """
    Filter out lines that are too dissimilar to the mean
    :param lines: list of lines
    :return: list of lines
    """

    data = np.array(lines)
    xs = data[:, 0] + data[:, 2]
    ys = data[:, 1] + data[:, 3]

    mean_equation = np.polyfit(xs, ys, deg=1)

    filtered_lines = []

    for line in lines:

        equation = np.polyfit([line[0], line[2]], [line[1], line[3]], deg=1)

        mean_angle = np.rad2deg(np.arctan(mean_equation[0]))
        line_angle = np.rad2deg(np.arctan(equation[0]))

        angular_difference = np.abs(mean_angle - line_angle)

        if angular_difference < 30:

            filtered_lines.append(line)

    return filtered_lines


def get_line_coordinates(lines):
    """
    Given a list of lines, returns a tuple (xs, ys), where xs are x coordinates and ys are y coordinates
    :param lines: list of lines, each line has (x1, y1, x2, y2) format
    :return: tuple (xs, ys)
    """

    xs = []
    ys = []

    for line in lines:

        x1, y1, x2, y2 = line

        xs.extend([x1, x2])
        ys.extend([y1, y2])

    return xs, ys


def get_lane_line(lines, image_shape):

    xs, ys = get_line_coordinates(lines)
    lane_equation = np.polyfit(xs, ys, deg=1)

    # Good value for input videos
    min_y = 320
    min_x = (min_y - lane_equation[1]) / lane_equation[0]

    max_y = image_shape[0]
    max_x = (max_y - lane_equation[1]) / lane_equation[0]

    lane = [int(coordinate) for coordinate in [min_x, min_y, max_x, max_y]]
    return lane


In [ ]:
import glob

# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
def pipeline(image):
    
    contours_image = get_xyz_space_contours(image)

    mask_vertices = get_simple_mask_vertices(image.shape)
    masked_image = region_of_interest(contours_image, mask_vertices)

    lines = cv2.HoughLinesP(
        masked_image, rho=4, theta=4 * math.pi / 180, threshold=100, lines=np.array([]),
        minLineLength=10, maxLineGap=10)

    # Hough lines are wrapped in unnecessary list, extract them for easier processing
    lines = [line[0] for line in lines]

    left_lines_candidates = get_left_road_lane_candidates(lines, image.shape)
    right_lines_candidates = get_right_road_lane_candidates(lines, image.shape)

    filtered_left_lines_candidates = get_filtered_lines(left_lines_candidates)
    filtered_right_lines_candidates = get_filtered_lines(right_lines_candidates)

    left_lane_line = get_lane_line(filtered_left_lines_candidates, image.shape)
    right_lane_line = get_lane_line(filtered_right_lines_candidates, image.shape)

    lane_lines = [left_lane_line, right_lane_line]

    # Wrap up lines to format expected by draw_lines
    lane_lines = [[line] for line in lane_lines]

    lanes_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    draw_lines(lanes_image, lane_lines, thickness=10, color=[255, 0, 0])
    draw_lane(lanes_image, left_lane_line, right_lane_line)
    lanes_overlay_image = weighted_img(lanes_image, image)

    return lanes_overlay_image


dirname = "test_images"
output_dir = os.path.join(dirname, "results")
os.makedirs(output_dir, exist_ok=True)

for path in glob.glob(os.path.join(dirname, "*.jpg")):
    
    image = mpimg.imread(path)
    lanes_image = pipeline(image)
                             
    output_path = os.path.join(output_dir, os.path.basename(path))
    cv2.imwrite(output_path, cv2.cvtColor(lanes_image, cv2.COLOR_RGB2BGR))


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    return pipeline(image)


Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

- Current algorithm is very much tailored to the videos with mask and filtering parameters that work well on given inputs, but won't necessary generalize well to all other inputs. It would be great to make a version that doesn't rely on these as much
- Lanes are approximated with lines, but they are really curves and curves could give better results
- Any glares in the camera would likely wreck havoc with results
- Algorith would get totally confused in parkings and other areas without lanes - essentially it assumes there always is a lane and as such picks up anything that looks like a lane line to it - in short anything that represents a line. A trailer standing in a parking lot at a right angle would be reported as a line
- I added code to draw green lane between found lane lines, since it looks really cool, but whenever one (or both) lane lines are not correctly found, plotted lane is a total rubbish
- in short algorithm has no knowledge of what constitutes a lane line - it just looks for lines that follow probable lane direction. It would be great to let it differentiate a line that has nothing to do with road markings from relevant lines/road features


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
# Code for challenge video processing

def get_challenge_mask_vertices(image_shape):

    return np.array([[
        (500, 450), (100, image_shape[0] - 50), (image_shape[1] - 100, image_shape[0] - 50), (image_shape[1] - 500, 450)
    ]])


def is_line_left_lane_candidate_challenge(line, image_shape):
    x1, y1, x2, y2 = line

    slope_degrees = np.rad2deg(np.arctan2(y2 - y1, x2 - x1))
    is_left_lane_slope = -45 < slope_degrees < -10

    max_x = 0.5 * image_shape[1]
    is_line_left_of_vehicle = x1 < max_x and max_x

    return is_left_lane_slope and is_line_left_of_vehicle


def is_line_right_lane_candidate_challenge(line, image_shape):
    x1, y1, x2, y2 = line

    slope_degrees = np.rad2deg(np.arctan2(y2 - y1, x2 - x1))
    is_right_lane_slope = 10 < slope_degrees < 45

    min_x = 0.5 * image_shape[1]
    is_line_right_of_vehicle = x1 > min_x and x2 > min_x

    return is_right_lane_slope and is_line_right_of_vehicle


def get_left_road_lane_candidates_challenge(lines, image_shape):

    return [line for line in lines if is_line_left_lane_candidate_challenge(line, image_shape)]


def get_right_road_lane_candidates_challenge(lines, image_shape):

    return [line for line in lines if is_line_right_lane_candidate_challenge(line, image_shape)]


def get_line_length(line):

    x1, y1, x2, y2 = line
    return np.sqrt((y2 - y1)**2 + (x2 - x1)**2)


def get_lines_in_descending_length_order(lines):

    line_lengths = [get_line_length(line) for line in lines]
    lines_lengths_tuple = zip(lines, line_lengths)

    # Sort lines by length in descending order
    sorted_lines_lengths_tuples = sorted(lines_lengths_tuple, key=lambda x: x[1], reverse=True)

    # Extract lines, now they are in descending length order
    return [line_length_tuple[0] for line_length_tuple in sorted_lines_lengths_tuples]


def are_lines_collinear(first, second):
    """
    Given two lines check if they are approximately collinear
    :param first: line
    :param second: line
    :return: boolean
    """

    first_equation = np.polyfit([first[0], first[2]], [first[1], first[3]], deg=1)
    second_equation = np.polyfit([second[0], second[2]], [second[1], second[3]], deg=1)

    first_angle = np.rad2deg(np.arctan(first_equation[0]))
    second_angle = np.rad2deg(np.arctan(second_equation[0]))

    angular_difference = np.abs(first_angle - second_angle)
    offset_distance = np.abs(first_equation[1] - second_equation[1])

    return angular_difference < 1 and offset_distance < 1


def merge_lines(first, second):

    xs = [first[0], first[2], second[0], second[2]]
    ys = [first[1], first[3], second[1], second[3]]

    min_index = np.argmin(xs)
    max_index = np.argmax(xs)

    return [xs[min_index], ys[min_index], xs[max_index], ys[max_index]]


def get_most_probable_lane(lines):

    sorted_lines = get_lines_in_descending_length_order(lines)

    lane_candidates = []
    lane_candidates_lengths = []

    for line in sorted_lines:

        is_collinear_line_found = False

        for index in range(len(lane_candidates)):

            if are_lines_collinear(lane_candidates[index], line):

                is_collinear_line_found = True
                lane_candidates[index] = merge_lines(lane_candidates[index], line)
                lane_candidates_lengths[index] += get_line_length(line)

        if not is_collinear_line_found:

            lane_candidates.append(line)
            lane_candidates_lengths.append(get_line_length(line))

    sorted_lane_candidates = get_lines_in_descending_length_order(lane_candidates)
    longest_lane = sorted_lane_candidates[0]

    return longest_lane


def get_lane_line_challenge(lines, image_shape):

    lane = get_most_probable_lane(lines)

    lane_equation = np.polyfit([lane[0], lane[2]], [lane[1], lane[3]], deg=1)

    min_y = 450
    min_x = (min_y - lane_equation[1]) / lane_equation[0]

    max_y = image_shape[0]
    max_x = (max_y - lane_equation[1]) / lane_equation[0]

    lane = [int(coordinate) for coordinate in [min_x, min_y, max_x, max_y]]
    return lane


def process_image_challenge(image):
    
    try:

        contours_image = get_xyz_space_contours(image)

        mask_vertices = get_challenge_mask_vertices(image.shape)
        masked_image = region_of_interest(contours_image, mask_vertices)

        lines = cv2.HoughLinesP(
            masked_image, rho=4, theta=math.pi / 180, threshold=100, lines=np.array([]),
            minLineLength=5, maxLineGap=20)

        # Hough lines are wrapped in unnecessary list, extract them for easier processing
        lines = [line[0] for line in lines]

        left_lines_candidates = get_left_road_lane_candidates_challenge(lines, image.shape)
        right_lines_candidates = get_right_road_lane_candidates_challenge(lines, image.shape)

        filtered_left_lines_candidates = get_filtered_lines(left_lines_candidates)
        filtered_right_lines_candidates = get_filtered_lines(right_lines_candidates)

        left_lane_line = get_lane_line_challenge(filtered_left_lines_candidates, image.shape)
        right_lane_line = get_lane_line_challenge(filtered_right_lines_candidates, image.shape)

        lane_lines = [left_lane_line, right_lane_line]

        # Wrap up lines to format expected by draw_lines
        lane_lines = [[line] for line in lane_lines]

        lanes_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
        draw_lines(lanes_image, lane_lines, thickness=10, color=[255, 0, 0])
        draw_lane(lanes_image, left_lane_line, right_lane_line)
        lanes_overlay_image = weighted_img(lanes_image, image)

        return lanes_overlay_image

    except:
        
        # Occasionally we fail to detect lines, return input image then
        return image


In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image_challenge)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))